In [33]:
import pandas as pd
import requests as r
from IPython.display import display
import numpy as np

In [34]:
pd.options.display.max_columns = None
pd. set_option('display.max_rows', None)

In [35]:
manga_data = []

In [36]:
for pages in [1,2,3,4,5,6,7,8]:
    url = 'https://api.jikan.moe/v4/top/manga?page='+str(pages)
    mal_manga = r.get(url)
    jmal_manga = mal_manga.json()
    manga_data = manga_data+jmal_manga['data']
    print(url)

https://api.jikan.moe/v4/top/manga?page=1
https://api.jikan.moe/v4/top/manga?page=2
https://api.jikan.moe/v4/top/manga?page=3
https://api.jikan.moe/v4/top/manga?page=4
https://api.jikan.moe/v4/top/manga?page=5
https://api.jikan.moe/v4/top/manga?page=6
https://api.jikan.moe/v4/top/manga?page=7
https://api.jikan.moe/v4/top/manga?page=8


In [37]:
dfm = pd.json_normalize(manga_data)

In [38]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   mal_id                       200 non-null    int64  
 1   url                          200 non-null    object 
 2   title                        200 non-null    object 
 3   title_english                152 non-null    object 
 4   title_japanese               200 non-null    object 
 5   title_synonyms               200 non-null    object 
 6   type                         200 non-null    object 
 7   chapters                     126 non-null    float64
 8   volumes                      127 non-null    float64
 9   status                       200 non-null    object 
 10  publishing                   200 non-null    bool   
 11  scored                       200 non-null    float64
 12  scored_by                    200 non-null    int64  
 13  rank                

In [39]:
cols = [0,1,3,4,5,6,10,12,14,15,16,17,18,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39]

In [40]:
dfm_new = dfm.drop(dfm.columns[cols],axis=1,inplace=False)

In [41]:
dfm_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           200 non-null    object 
 1   chapters        126 non-null    float64
 2   volumes         127 non-null    float64
 3   status          200 non-null    object 
 4   scored          200 non-null    float64
 5   rank            200 non-null    int64  
 6   authors         200 non-null    object 
 7   serializations  200 non-null    object 
 8   genres          200 non-null    object 
 9   demographics    200 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 15.8+ KB


In [42]:
missing_mask = dfm_new['chapters'].isna() 
# Finds all NaN values in 'chapters' column and determines whether there are NaN values (boolean T/F)
mapping_dict = dict({'Publishing': 'Ongoing'})
# creates a dicitionary which finds all 'publishing' values and will allow for those values to be replaced to 'Ongoing'
dfm_new.loc[missing_mask,'chapters'] = dfm_new.loc[missing_mask,'status'].map(mapping_dict)
# creates a slice of the 'chapters' column which have NaN values and creates a loop with the map command 
# to go through the entire column
dfm_new['chapters'] = dfm_new['chapters'].fillna('Ongoing')
# uses the fillna command to fill all NaN values with 'Ongoing'

dfm_new.loc[dfm_new['volumes'].isna(),'volumes'] = dfm_new.loc[dfm_new['volumes'].isna()]['chapters']
# Finds all NaN values in the 'volumes' column and changes them to be equal to the values
# in the same row in the 'chapters column'

In [43]:
dfm_new['serializations'][0]
# looks at the first value in the 'serialization' column

[{'mal_id': 2,
  'type': 'manga',
  'name': 'Young Animal',
  'url': 'https://myanimelist.net/manga/magazine/2/Young_Animal'}]

In [44]:
dfm_new['authors'] = dfm_new['authors'].map(lambda authors: [author['name'] for author in authors])
# takes the first value of the 'authors' column and creates a map 
# similar to the above one where it loops through the entire column.
# the map allows for a function which uses a custom function (lambda) to take only the author's name in the dictionary

In [45]:
dfm_new['serializations'] = dfm_new['serializations'].map(lambda serializations: [serialization['name'] for serialization in serializations])

In [46]:
dfm_new['genres'] = dfm_new['genres'].map(lambda genres: [genre['name'] for genre in genres])

In [47]:
dfm_new['demographics'] = dfm_new['demographics'].map(lambda demographics: [demographic['name'] for demographic in demographics])

In [48]:
# dfm_new.to_excel(r'C:\Users\hirok\OneDrive\Documents\Data Analysis\MAL Project\dfm_new.xlsx',index = False, header = True)

In [49]:
dfm_new.head()

,title,chapters,volumes,status,scored,rank,authors,serializations,genres,demographics
0,Berserk,380.0,41.0,On Hiatus,9.43,1,"[Miura, Kentarou]",[Young Animal],"[Action, Adventure, Drama, Fantasy, Horror, Su...",[Seinen]
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,96.0,24.0,Finished,9.27,2,"[Araki, Hirohiko]",[Ultra Jump],"[Action, Adventure, Horror, Mystery, Supernatu...","[Seinen, Shounen]"
2,One Piece,Ongoing,Ongoing,Publishing,9.17,3,"[Oda, Eiichiro]",[Shounen Jump (Weekly)],"[Action, Adventure, Comedy, Fantasy]",[Shounen]
3,Vagabond,327.0,37.0,On Hiatus,9.16,4,"[Inoue, Takehiko, Yoshikawa, Eiji]",[Morning],"[Action, Adventure, Drama]",[Seinen]
4,Monster,162.0,18.0,Finished,9.12,5,"[Urasawa, Naoki]",[Big Comic Original],"[Drama, Mystery]",[Seinen]


In [50]:
dfm_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           200 non-null    object 
 1   chapters        200 non-null    object 
 2   volumes         200 non-null    object 
 3   status          200 non-null    object 
 4   scored          200 non-null    float64
 5   rank            200 non-null    int64  
 6   authors         200 non-null    object 
 7   serializations  200 non-null    object 
 8   genres          200 non-null    object 
 9   demographics    200 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 15.8+ KB


In [51]:
colz = [1,2,3,4,6,7]

In [52]:
dfm_use = dfm_new.drop(dfm_new.columns[colz],axis=1,inplace=False)

In [53]:
dfm_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         200 non-null    object
 1   rank          200 non-null    int64 
 2   genres        200 non-null    object
 3   demographics  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [54]:
dfm_use.head()

,title,rank,genres,demographics
0,Berserk,1,"[Action, Adventure, Drama, Fantasy, Horror, Su...",[Seinen]
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Run,2,"[Action, Adventure, Horror, Mystery, Supernatu...","[Seinen, Shounen]"
2,One Piece,3,"[Action, Adventure, Comedy, Fantasy]",[Shounen]
3,Vagabond,4,"[Action, Adventure, Drama]",[Seinen]
4,Monster,5,"[Drama, Mystery]",[Seinen]


In [55]:
dfm_use['genres'][0]

['Action', 'Adventure', 'Drama', 'Fantasy', 'Horror', 'Supernatural']

In [56]:
dfm_use.to_excel(r'C:\Users\hirok\OneDrive\Documents\Data Analysis\MAL Project\dfm_use.xlsx',index = False, header = True)